In [1]:
# Required Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, classification_report,confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, auc, roc_curve
import pickle
import logging
import sys
import optuna
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load Dataset
data = pd.read_csv("../Data/preprocessed_data")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9730347 entries, 0 to 9730346
Data columns (total 30 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Unnamed: 0                      int64  
 1   order_id                        int64  
 2   product_id                      int64  
 3   add_to_cart_order               int64  
 4   reordered                       int64  
 5   user_id                         int64  
 6   order_number                    int64  
 7   days_since_prior_order          float64
 8   average_days_between_purchases  float64
 9   product_purchase_frequency      int64  
 10  total_purchases                 int64  
 11  interval_std_dev                float64
 12  product_reorder_rate            float64
 13  users_general_reorder_rate      float64
 14  avg_add_to_cart_order           float64
 15  order_frequency_by_dow          int64  
 16  aisle_target_enc                float64
 17  department_target_enc      

In [5]:
# Data Preview
data.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,average_days_between_purchases,product_purchase_frequency,...,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,order_hour_sin,order_hour_cos,add_to_cart_order_encoded,days_since_prior_order_binned
0,0,2722718,8619,9,0,41627,5,15.0,15.000000,1,...,1,0,0,0,0,0,2.588190e-01,-0.965926,8.0,8-15
1,1,2089674,13870,10,0,146296,28,29.0,29.000000,1,...,0,0,0,1,0,0,8.660254e-01,-0.500000,9.0,24-31
2,2,3024155,2029,1,0,6200,2,4.0,4.000000,1,...,0,0,0,0,1,0,7.071068e-01,-0.707107,0.0,0-7
3,3,2890872,16062,2,0,8337,32,2.0,2.000000,1,...,0,0,0,0,1,0,1.224647e-16,-1.000000,1.0,0-7
4,4,1798802,14335,3,0,155167,1,0.0,7.166667,6,...,0,1,0,0,0,0,-9.659258e-01,-0.258819,2.0,Unknown


In [6]:
data = data.drop(columns=("Unnamed: 0"))

In [7]:
data.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,average_days_between_purchases,product_purchase_frequency,total_purchases,...,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,order_hour_sin,order_hour_cos,add_to_cart_order_encoded,days_since_prior_order_binned
0,2722718,8619,9,0,41627,5,15.0,15.000000,1,1,...,1,0,0,0,0,0,2.588190e-01,-0.965926,8.0,8-15
1,2089674,13870,10,0,146296,28,29.0,29.000000,1,1,...,0,0,0,1,0,0,8.660254e-01,-0.500000,9.0,24-31
2,3024155,2029,1,0,6200,2,4.0,4.000000,1,1,...,0,0,0,0,1,0,7.071068e-01,-0.707107,0.0,0-7
3,2890872,16062,2,0,8337,32,2.0,2.000000,1,1,...,0,0,0,0,1,0,1.224647e-16,-1.000000,1.0,0-7
4,1798802,14335,3,0,155167,1,0.0,7.166667,6,6,...,0,1,0,0,0,0,-9.659258e-01,-0.258819,2.0,Unknown


In [8]:
data['reordered'].unique()

array([0, 1])

In [9]:
data['reordered'].value_counts()

reordered
1    5737961
0    3992386
Name: count, dtype: int64

# Selected Model - LightGBM & The Hyperparameters to consider

## Learning Parameters
* **learning_rate** - A smaller value may increas overall performance but will require more iterations
* **n_estimators** - Number of boosting rounds
* **num_leaves** - Maximum number of leaves in a tree. Larger values increase accuracy but risk overfitting
* **max_depth** - Maximum Depth of a tree
* **min_data_in_leaf** - Minimum number of samples per leaf. Higher Values prevent overfitting

## Regularization Parameters
* **lambda_l1** - L1 regularization term on weights. Adds sparsity to the model
* **lambda_l2** - L2 regularization term on weights. Helps control overfitting
* **min_gain_to_split** - Minimum loss reduction required to split a leaf node. Larger values make the model training faster
* **early_stopping_round** - Helps stop training when performance does not improve

## Data Sampling Parameters
* **bagging_fraction** - Fraction of data used for each iteration (for bagging). Helps with overfitting
* **bagging_freq** - Frequency of bagging. Also helps deal with overfittiing

### Hyperparameter tuning Technique used is Bayesian Optimization Technique

In [10]:
#Train Test Split
data = data.drop(columns=(["order_id","product_id","user_id","days_since_prior_order_binned"]))
X = data.drop(columns="reordered")
y = data["reordered"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print("X_train Shape:",X_train.shape)
print("X_test Shape:",X_test.shape)
print("y_train Shape:", y_train.shape)
print("y_test Shape:",y_test.shape)

X_train Shape: (7784277, 24)
X_test Shape: (1946070, 24)
y_train Shape: (7784277,)
y_test Shape: (1946070,)


In [4]:
# ObJective Function which will be evaluated 
def objective_function(trial):
    # Hyperparameter Search Space
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-4, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-4, 10.0),
        "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
    }
    print(param_grid)
    model = lightgbm.LGBMClassifier(**param_grid, n_jobs=1, learning_rate=0.01, random_state=42)
    model.fit(X_train.fillna(0),y_train)
    y_pred = model.predict(X_test.fillna(0))
    
    return f1_score(y_test,y_pred)



**Day 1 of Hyperparameter tuning**

In [ ]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "purchase_hp_tuning"
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.create_study(study_name = study_name, storage=storage_name, load_if_exists=True, direction='maximize')
study.optimize(objective_function, n_trials=50, n_jobs=2)


[I 2024-11-21 12:40:30,276] A new study created in RDB with name: purchase_hp_tuning


A new study created in RDB with name: purchase_hp_tuning
{'n_estimators': 493, 'num_leaves': 88, 'max_depth': 8, 'min_data_in_leaf': 32, 'lambda_l1': 0.0003197464313668558, 'lambda_l2': 0.3148857374630904, 'min_gain_to_split': 0.047546251367120755, 'bagging_fraction': 0.7560656997649566, 'bagging_freq': 1}
{'n_estimators': 399, 'num_leaves': 51, 'max_depth': 0, 'min_data_in_leaf': 41, 'lambda_l1': 0.2878043360786342, 'lambda_l2': 0.008851088544823778, 'min_gain_to_split': 0.003415565231852756, 'bagging_fraction': 0.7681675131542959, 'bagging_freq': 5}
[LightGBM] [Warning] lambda_l1 is set=0.0003197464313668558, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0003197464313668558
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.3148857374630904, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3148857374630904
[LightGBM] [Warning] bagging_fraction is set=0.75606569976495

[I 2024-11-21 12:49:29,585] Trial 0 finished with value: 0.8466903830583237 and parameters: {'n_estimators': 493, 'num_leaves': 88, 'max_depth': 8, 'min_data_in_leaf': 32, 'lambda_l1': 0.0003197464313668558, 'lambda_l2': 0.3148857374630904, 'min_gain_to_split': 0.047546251367120755, 'bagging_fraction': 0.7560656997649566, 'bagging_freq': 1}. Best is trial 0 with value: 0.8466903830583237.


Trial 0 finished with value: 0.8466903830583237 and parameters: {'n_estimators': 493, 'num_leaves': 88, 'max_depth': 8, 'min_data_in_leaf': 32, 'lambda_l1': 0.0003197464313668558, 'lambda_l2': 0.3148857374630904, 'min_gain_to_split': 0.047546251367120755, 'bagging_fraction': 0.7560656997649566, 'bagging_freq': 1}. Best is trial 0 with value: 0.8466903830583237.
{'n_estimators': 275, 'num_leaves': 45, 'max_depth': 1, 'min_data_in_leaf': 14, 'lambda_l1': 0.02542367993824871, 'lambda_l2': 0.0022976806895551975, 'min_gain_to_split': 0.0025181238264553817, 'bagging_fraction': 0.8893307966155001, 'bagging_freq': 2}
[LightGBM] [Warning] lambda_l1 is set=0.02542367993824871, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.02542367993824871
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.0022976806895551975, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0022976806895551975


[I 2024-11-21 12:50:27,576] Trial 1 finished with value: 0.8465953782461692 and parameters: {'n_estimators': 399, 'num_leaves': 51, 'max_depth': 0, 'min_data_in_leaf': 41, 'lambda_l1': 0.2878043360786342, 'lambda_l2': 0.008851088544823778, 'min_gain_to_split': 0.003415565231852756, 'bagging_fraction': 0.7681675131542959, 'bagging_freq': 5}. Best is trial 0 with value: 0.8466903830583237.


...

Trial 34 finished with value: 0.8471259310545065 and parameters: {'n_estimators': 485, 'num_leaves': 98, 'max_depth': 9, 'min_data_in_leaf': 31, 'lambda_l1': 0.0002056362076497198, 'lambda_l2': 0.010452132525854979, 'min_gain_to_split': 0.01774293615061127, 'bagging_fraction': 0.7790527376033966, 'bagging_freq': 4}. Best is trial 32 with value: 0.8473495380911816.
{'n_estimators': 391, 'num_leaves': 91, 'max_depth': 9, 'min_data_in_leaf': 31, 'lambda_l1': 0.0005559699391079436, 'lambda_l2': 0.0038056082825090654, 'min_gain_to_split': 0.05994535690662769, 'bagging_fraction': 0.8477176637574408, 'bagging_freq': 5}
[LightGBM] [Warning] lambda_l1 is set=0.0005559699391079436, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0005559699391079436
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.0038056082825090654, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0038056082825

[I 2024-11-21 15:09:16,875] Trial 35 finished with value: 0.8471333369066123 and parameters: {'n_estimators': 484, 'num_leaves': 100, 'max_depth': 9, 'min_data_in_leaf': 31, 'lambda_l1': 0.0005194573920006524, 'lambda_l2': 0.003496296782240002, 'min_gain_to_split': 0.06339161950714192, 'bagging_fraction': 0.7766216783151613, 'bagging_freq': 4}. Best is trial 32 with value: 0.8473495380911816.


Trial 35 finished with value: 0.8471333369066123 and parameters: {'n_estimators': 484, 'num_leaves': 100, 'max_depth': 9, 'min_data_in_leaf': 31, 'lambda_l1': 0.0005194573920006524, 'lambda_l2': 0.003496296782240002, 'min_gain_to_split': 0.06339161950714192, 'bagging_fraction': 0.7766216783151613, 'bagging_freq': 4}. Best is trial 32 with value: 0.8473495380911816.
{'n_estimators': 414, 'num_leaves': 92, 'max_depth': 8, 'min_data_in_leaf': 36, 'lambda_l1': 0.0008407971367883483, 'lambda_l2': 0.010524297459055157, 'min_gain_to_split': 0.14266946264219435, 'bagging_fraction': 0.8510250566001288, 'bagging_freq': 5}
[LightGBM] [Warning] lambda_l1 is set=0.0008407971367883483, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0008407971367883483
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.010524297459055157, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01052429745905

[I 2024-11-21 15:12:51,607] Trial 36 finished with value: 0.8465263090087285 and parameters: {'n_estimators': 391, 'num_leaves': 91, 'max_depth': 9, 'min_data_in_leaf': 31, 'lambda_l1': 0.0005559699391079436, 'lambda_l2': 0.0038056082825090654, 'min_gain_to_split': 0.05994535690662769, 'bagging_fraction': 0.8477176637574408, 'bagging_freq': 5}. Best is trial 32 with value: 0.8473495380911816.


Trial 36 finished with value: 0.8465263090087285 and parameters: {'n_estimators': 391, 'num_leaves': 91, 'max_depth': 9, 'min_data_in_leaf': 31, 'lambda_l1': 0.0005559699391079436, 'lambda_l2': 0.0038056082825090654, 'min_gain_to_split': 0.05994535690662769, 'bagging_fraction': 0.8477176637574408, 'bagging_freq': 5}. Best is trial 32 with value: 0.8473495380911816.
{'n_estimators': 422, 'num_leaves': 82, 'max_depth': 8, 'min_data_in_leaf': 27, 'lambda_l1': 0.004790446752160439, 'lambda_l2': 0.011536608587824303, 'min_gain_to_split': 0.20292100734405055, 'bagging_fraction': 0.8488399094680075, 'bagging_freq': 3}
[LightGBM] [Warning] lambda_l1 is set=0.004790446752160439, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.004790446752160439
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=0.011536608587824303, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01153660858782430

In [15]:
# Best trial
print("Best trial parameters:", study.best_params)
print("Best trial score:", study.best_value)

Best trial parameters: {'n_estimators': 472, 'num_leaves': 100, 'max_depth': 10, 'min_data_in_leaf': 35, 'lambda_l1': 0.00022263396207797137, 'lambda_l2': 0.03404409194756445, 'min_gain_to_split': 0.03997431193169202, 'bagging_fraction': 0.8147245992254589, 'bagging_freq': 4}
Best trial score: 0.8473495380911816


**Day 2 of Hyperparameter Tuning**

In [12]:
study_name = "purchase_hp_tuning"
storage_name = "sqlite:///{}.db".format(study_name)

study1 = optuna.load_study(study_name=study_name, storage=storage_name)
print("Best trial parameters:", study1.best_params)
print("Best trial score:", study1.best_value)

Best trial parameters: {'n_estimators': 472, 'num_leaves': 100, 'max_depth': 10, 'min_data_in_leaf': 35, 'lambda_l1': 0.00022263396207797137, 'lambda_l2': 0.03404409194756445, 'min_gain_to_split': 0.03997431193169202, 'bagging_fraction': 0.8147245992254589, 'bagging_freq': 4}
Best trial score: 0.8473495380911816


In [13]:
study1.optimize(objective_function, n_trials=50, n_jobs=2)

{'n_estimators': 580, 'num_leaves': 128, 'max_depth': 13, 'min_data_in_leaf': 66, 'lambda_l1': 9.818222835036108, 'lambda_l2': 0.9108136174317132, 'min_gain_to_split': 0.007381564861038712, 'bagging_fraction': 0.6290078644213785, 'bagging_freq': 6}
{'n_estimators': 581, 'num_leaves': 136, 'max_depth': 13, 'min_data_in_leaf': 62, 'lambda_l1': 0.016734542565462994, 'lambda_l2': 0.6168662711899038, 'min_gain_to_split': 0.006741367107895389, 'bagging_fraction': 0.9005260047576603, 'bagging_freq': 6}
[LightGBM] [Warning] lambda_l1 is set=0.016734542565462994, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.016734542565462994
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=0.6168662711899038, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6168662711899038
[LightGBM] [Warning] bagging_fraction is set=0.9005260047576603, subsample=1.0 will be ignored. Current value: bagging_f

[I 2024-11-29 10:53:22,995] Trial 50 finished with value: 0.8481961160694583 and parameters: {'n_estimators': 580, 'num_leaves': 128, 'max_depth': 13, 'min_data_in_leaf': 66, 'lambda_l1': 9.818222835036108, 'lambda_l2': 0.9108136174317132, 'min_gain_to_split': 0.007381564861038712, 'bagging_fraction': 0.6290078644213785, 'bagging_freq': 6}. Best is trial 50 with value: 0.8481961160694583.


{'n_estimators': 587, 'num_leaves': 138, 'max_depth': 13, 'min_data_in_leaf': 64, 'lambda_l1': 0.06339283716206691, 'lambda_l2': 4.310763423562925, 'min_gain_to_split': 0.006156028802268479, 'bagging_fraction': 0.6425911214891878, 'bagging_freq': 7}
[LightGBM] [Warning] lambda_l1 is set=0.06339283716206691, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06339283716206691
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l2 is set=4.310763423562925, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.310763423562925
[LightGBM] [Warning] bagging_fraction is set=0.6425911214891878, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6425911214891878
[LightGBM] [Warning] min_data_in_leaf is set=64, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=64
[LightGBM] [Warning] min_gain_to_split is set=0.006156028802268479, min_split_gain=0.0 will be ignored. Cu

[I 2024-11-29 10:55:29,393] Trial 51 finished with value: 0.848234670480591 and parameters: {'n_estimators': 581, 'num_leaves': 136, 'max_depth': 13, 'min_data_in_leaf': 62, 'lambda_l1': 0.016734542565462994, 'lambda_l2': 0.6168662711899038, 'min_gain_to_split': 0.006741367107895389, 'bagging_fraction': 0.9005260047576603, 'bagging_freq': 6}. Best is trial 51 with value: 0.848234670480591.


{'n_estimators': 577, 'num_leaves': 127, 'max_depth': 13, 'min_data_in_leaf': 66, 'lambda_l1': 7.332015954838193, 'lambda_l2': 5.084231068986714, 'min_gain_to_split': 0.005673102094047818, 'bagging_fraction': 0.6658263654260979, 'bagging_freq': 8}
[LightGBM] [Warning] lambda_l1 is set=7.332015954838193, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.332015954838193
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
[LightGBM] [Warning] lambda_l2 is set=5.084231068986714, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.084231068986714
[LightGBM] [Warning] bagging_fraction is set=0.6658263654260979, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6658263654260979
[LightGBM] [Warning] min_data_in_leaf is set=66, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=66
[LightGBM] [Warning] min_gain_to_split is set=0.005673102094047818, min_split_gain=0.0 will be ignored. Current 

...

[I 2024-11-29 16:14:16,534] Trial 76 finished with value: 0.8488604349741958 and parameters: {'n_estimators': 930, 'num_leaves': 200, 'max_depth': 12, 'min_data_in_leaf': 98, 'lambda_l1': 0.5891322610259099, 'lambda_l2': 1.8126635148007308, 'min_gain_to_split': 0.22757515405598033, 'bagging_fraction': 0.6893543920321131, 'bagging_freq': 9}. Best is trial 73 with value: 0.8490418110706236.


{'n_estimators': 966, 'num_leaves': 190, 'max_depth': 11, 'min_data_in_leaf': 93, 'lambda_l1': 0.3947363797713747, 'lambda_l2': 1.3825414480066722, 'min_gain_to_split': 0.2581538075953894, 'bagging_fraction': 0.6886740991589452, 'bagging_freq': 9}
[LightGBM] [Warning] lambda_l1 is set=0.3947363797713747, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3947363797713747
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] lambda_l2 is set=1.3825414480066722, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3825414480066722
[LightGBM] [Warning] bagging_fraction is set=0.6886740991589452, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6886740991589452
[LightGBM] [Warning] min_data_in_leaf is set=93, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=93
[LightGBM] [Warning] min_gain_to_split is set=0.2581538075953894, min_split_gain=0.0 will be ignored. Curren

[I 2024-11-29 16:27:20,279] Trial 77 finished with value: 0.8489227993183261 and parameters: {'n_estimators': 956, 'num_leaves': 200, 'max_depth': 12, 'min_data_in_leaf': 96, 'lambda_l1': 0.5486605773722949, 'lambda_l2': 1.6462310434939174, 'min_gain_to_split': 0.22819265147730255, 'bagging_fraction': 0.6869936115417846, 'bagging_freq': 9}. Best is trial 73 with value: 0.8490418110706236.


{'n_estimators': 971, 'num_leaves': 190, 'max_depth': 14, 'min_data_in_leaf': 92, 'lambda_l1': 0.3439142698852568, 'lambda_l2': 1.3288318289273937, 'min_gain_to_split': 0.24035195998981623, 'bagging_fraction': 0.699417894976562, 'bagging_freq': 9}
[LightGBM] [Warning] lambda_l1 is set=0.3439142698852568, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3439142698852568
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] lambda_l2 is set=1.3288318289273937, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.3288318289273937
[LightGBM] [Warning] bagging_fraction is set=0.699417894976562, subsample=1.0 will be ignored. Current value: bagging_fraction=0.699417894976562
[LightGBM] [Warning] min_data_in_leaf is set=92, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=92
[LightGBM] [Warning] min_gain_to_split is set=0.24035195998981623, min_split_gain=0.0 will be ignored. Current

KeyboardInterrupt: 

In [17]:
# Best trial
print("Best trial parameters:", study1.best_params)
print("Best trial score:", study1.best_value)

Best trial parameters: {'n_estimators': 913, 'num_leaves': 198, 'max_depth': 14, 'min_data_in_leaf': 98, 'lambda_l1': 0.8545888850259123, 'lambda_l2': 1.9370204406076479, 'min_gain_to_split': 0.29200229817882095, 'bagging_fraction': 0.6785186169854743, 'bagging_freq': 9}
Best trial score: 0.8490418110706236


In [2]:
study_name = "purchase_hp_tuning"
storage_name = "sqlite:///{}.db".format(study_name)

study2 = optuna.load_study(study_name=study_name, storage=storage_name)
print("Best trial parameters:", study2.best_params)
print("Best trial score:", study2.best_value)

Best trial parameters: {'n_estimators': 971, 'num_leaves': 190, 'max_depth': 14, 'min_data_in_leaf': 92, 'lambda_l1': 0.3439142698852568, 'lambda_l2': 1.3288318289273937, 'min_gain_to_split': 0.24035195998981623, 'bagging_fraction': 0.699417894976562, 'bagging_freq': 9}
Best trial score: 0.8490936029671841


In [3]:
len(study2.trials)

80